In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import os
import csv
import re
from time import sleep

In [2]:
def site_login():
    """
    셀레니움을 통해 사이트에 로그인을 한다.
    """
    driver.get ("http://www.catcare.or.kr/index.php?act=dispMemberLoginForm")
    driver.find_element_by_id("uemail").send_keys("snorlax")
    driver.find_element_by_id ("upw").send_keys("rescuecat1212")
    #driver.find_element_by_css_selector("#xe_popup2704778 > div.popupcloser > button").click()
    driver.find_element_by_xpath("//*[@id=\"cont_body\"]/div/div[2]/div[2]/div/section/section/div[2]/div[2]/form/fieldset/div[2]/button").click()

In [3]:
def save_to_csv(data):
#최초 생성 시
    if os.path.exists("./tnr_cat.csv")==False:
        #헤더를 저장한다.
        with open ("./tnr_cat.csv", "w", encoding='utf-8-sig',newline="") as fp:
            writer=csv.writer(fp)
            writer.writerow(["no", "date", "title", "author", "view"])
        
    #데이터 저장
    with open("./tnr_cat.csv", 'a', encoding='utf-8-sig',newline="") as fp2:
        writer2=csv.writer(fp2)
        writer2.writerow(data)

In [4]:
#셀레니움 실행
driver = webdriver.Chrome("./chromedriver.exe")
#로그인한다.
site_login()

#반복하며 가져오기 위한 변수들을 준비한다.

#게시판으로 이동
driver.get ("http://www.catcare.or.kr/index.php?mid=actitnr&category=279")

#게시판의 마지막 페이지
last_page=int(driver.find_element_by_xpath("//*[@id=\"bd_277_0\"]/div/form/fieldset/a[11]").text)

#현재 페이지 카운터
cur_page=1

In [5]:
while True:    
    #길냥이사랑 TNR 신청 게시판으로 넘어간다.
    driver.get ("http://www.catcare.or.kr/index.php?mid=actitnr&category=279&page="+str(cur_page))
    sleep(1)
    
    #게시판의 총 게시글 수
    row_count = len(driver.find_elements_by_xpath("//*[@id=\"bd_277_0\"]/div/table/tbody/tr"))
    
    #게시판의 총 공지글 수
    notice_count=len(driver.find_elements_by_xpath("//*[contains(@class,'notice')]"))
    
    #반복문 범위
    iter_range=(row_count-notice_count)+2
    
    #게시판을 파싱한다.
    soup=BeautifulSoup(driver.page_source, 'html.parser')
    
    
    if cur_page <= last_page:
        for i in range(2,iter_range):
            #게시글의 번호            
            no=driver.find_element_by_xpath("//*[@id=\"bd_277_0\"]/div/table/tbody/tr["+str(i)+"]/td[1]").text
            #게시글 작성 날짜
            date=re.sub("\.", "-",driver.find_element_by_xpath("//*[@id=\"bd_277_0\"]/div/table/tbody/tr["+str(i)+"]/td[5]").text)
            #제목
            title=' '.join(re.sub("\W", " ", driver.find_element_by_xpath("//*[@id=\"bd_277_0\"]/div/table/tbody/tr["+str(i)+"]/td[3]/a").text).split())
            #글쓴이
            author=driver.find_element_by_xpath("//*[@id=\"bd_277_0\"]/div/table/tbody/tr["+str(i)+"]/td[4]/span/a").text
            #조회수
            view=driver.find_element_by_xpath("//*[@id=\"bd_277_0\"]/div/table/tbody/tr["+str(i)+"]/td[6]").text
            
            #리스트에 담는다.
            data_list=[no, date, title, author, view]

            save_to_csv(data_list)
            
        cur_page+=1
        
    else:
        break;
driver.close()